In [79]:
#Imports and set up
import os, math, json, random, warnings
import numpy as np
import pandas as pd

from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')


from chembl_webresource_client.new_client import new_client

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
import optuna

from xgboost import XGBRegressor

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader


In [80]:
# Clean output
warnings.filterwarnings("ignore")
pd.set_option("display.max_colwidth", 120)

# --- Fix seeds for reproducibility ---
import torch

SEED = 42

# Python & NumPy
random.seed(SEED)
np.random.seed(SEED)

# PyTorch (CPU & GPU/MPS)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

# Ensure deterministic behavior (slower but reproducible)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True, warn_only=True)

# 🔹 Use a seeded generator for DataLoader shuffling
g = torch.Generator().manual_seed(SEED)

In [81]:
#Parameters
target   = "CHEMBL203"   # EGFR
endpoint = "IC50"
outdir   = "outputs/egfr_ic50_scaffold"
os.makedirs(outdir, exist_ok=True)

FP_BITS   = 1024
FP_RADIUS = 2

In [82]:
#Helpers
UNIT_TO_M = {"M":1,"mM":1e-3,"uM":1e-6,"µM":1e-6,"nM":1e-9,"pM":1e-12}

def to_molar(val, units):
    try: return float(val) * UNIT_TO_M.get(units, np.nan)
    except: return None

def to_pIC50(molar):
    return -math.log10(molar) if molar and molar > 0 else None

def clean_smiles(smi):
    mol = Chem.MolFromSmiles(smi) if isinstance(smi,str) else None
    if mol is None: return None
    lfc = rdMolStandardize.LargestFragmentChooser(preferOrganic=True)
    mol = lfc.choose(mol)
    mol = rdMolStandardize.Uncharger().uncharge(mol)
    try: Chem.SanitizeMol(mol)
    except: return None
    return Chem.MolToSmiles(mol, canonical=True)

def scaffold_split(df, frac_train=0.8, seed=42):
    """Split dataset into train/test by Bemis–Murcko scaffolds."""
    scaffolds = {}
    for idx, smi in enumerate(df["clean_smiles"]):
        mol = Chem.MolFromSmiles(smi)
        scaffold = MurckoScaffold.MurckoScaffoldSmiles(mol=mol, includeChirality=False)
        scaffolds.setdefault(scaffold, []).append(idx)

    # sort by scaffold size
    sorted_scaffolds = sorted(scaffolds.items(), key=lambda x: len(x[1]), reverse=True)
    rng = random.Random(seed)
    rng.shuffle(sorted_scaffolds)

    n_total = len(df)
    n_train = int(frac_train * n_total)
    train_idx, test_idx = [], []

    for scaffold, idxs in sorted_scaffolds:
        if len(train_idx) + len(idxs) <= n_train:
            train_idx.extend(idxs)
        else:
            test_idx.extend(idxs)

    return df.iloc[train_idx], df.iloc[test_idx]

In [83]:
#Fetch ChEMBL data
print("📥 Fetching data...")
acts = new_client.activity.filter(target_chembl_id=target, standard_type=endpoint)
# keep only high-confidence assays
acts = acts.filter(assay_confidence_score__gte=8)  
rows = [{"molecule_chembl_id":r.get("molecule_chembl_id"),
         "canonical_smiles":r.get("canonical_smiles"),
         "standard_value":r.get("standard_value"),
         "standard_units":r.get("standard_units"),
         "standard_relation":r.get("standard_relation")} for r in acts]
df = pd.DataFrame(rows)
print("Raw shape:", df.shape)

📥 Fetching data...
Raw shape: (18105, 5)


In [84]:
#Cleaning and transformation
df = df[df["standard_relation"].isin(["=","~","≈",None])]
df["clean_smiles"] = df["canonical_smiles"].apply(clean_smiles)
df["molar"] = df.apply(lambda r: to_molar(r["standard_value"], r["standard_units"]), axis=1)
df["pIC50"] = df["molar"].apply(to_pIC50)
df = df.dropna(subset=["clean_smiles","pIC50"]).reset_index(drop=True)
# 🔹 aggregate duplicates: median pIC50 per molecule
df = df.groupby(["molecule_chembl_id", "clean_smiles"], as_index=False)["pIC50"].median()
print("After cleaning:", df.shape)
df.to_csv(os.path.join(outdir,"processed.csv"), index=False)

After cleaning: (7909, 3)


In [85]:
#Featurisation (Fingerprint and descriptors)
print("🔬 Featurizing molecules...")
mols = [Chem.MolFromSmiles(s) for s in df["clean_smiles"]]

# Morgan FP
gen = GetMorganGenerator(radius=FP_RADIUS, fpSize=FP_BITS)
def fp_array(m): 
    arr = np.zeros((FP_BITS,), dtype=int)
    if m: Chem.DataStructs.ConvertToNumpyArray(gen.GetFingerprint(m), arr)
    return arr
fps = np.array([fp_array(m) for m in mols], dtype=int)

# RDKit descriptors
desc_funcs = [f for _,f in Descriptors._descList]
def desc_array(m):
    return np.array([f(m) for f in desc_funcs], dtype=float) if m else np.zeros((len(desc_funcs),))
desc = np.array([desc_array(m) for m in mols], dtype=float)

# Combine
X = np.hstack([fps, desc])
y = df["pIC50"].values.astype(float)

imputer = SimpleImputer(strategy="constant", fill_value=0.0)
X = imputer.fit_transform(X)

print("Feature matrix:", X.shape)

🔬 Featurizing molecules...
Feature matrix: (7909, 1241)


In [86]:
#Classical Models
metrics = {}

# --- Scaffold-based K-Fold Cross Validation ---
def scaffold_kfold(df, X, y, n_splits=5, seed=42):
    scaffolds = {}
    for idx, smi in enumerate(df["clean_smiles"]):
        mol = Chem.MolFromSmiles(smi)
        scaffold = MurckoScaffold.MurckoScaffoldSmiles(mol=mol, includeChirality=False)
        scaffolds.setdefault(scaffold, []).append(idx)

    all_scaffolds = list(scaffolds.values())
    rng = np.random.default_rng(seed)
    rng.shuffle(all_scaffolds)

    folds = [[] for _ in range(n_splits)]
    for i, group in enumerate(all_scaffolds):
        folds[i % n_splits].extend(group)

    for k in range(n_splits):
        test_idx = folds[k]
        train_idx = [i for j in range(n_splits) if j != k for i in folds[j]]
        yield train_idx, test_idx

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# --- RandomForest CV ---
rf_rmse, rf_mae, rf_r2 = [], [], []
for train_idx, test_idx in scaffold_kfold(df, X, y, n_splits=5):
    rf = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1)
    rf.fit(X[train_idx], y[train_idx])
    pred = rf.predict(X[test_idx])
    rf_rmse.append(np.sqrt(mean_squared_error(y[test_idx], pred)))
    rf_mae.append(mean_absolute_error(y[test_idx], pred))
    rf_r2.append(r2_score(y[test_idx], pred))
metrics["RandomForest_CV"] = {
    "RMSE": np.mean(rf_rmse),
    "MAE": np.mean(rf_mae),
    "R2": np.mean(rf_r2)
}

# --- SVR CV ---
svr_rmse, svr_mae, svr_r2 = [], [], []
for train_idx, test_idx in scaffold_kfold(df, X, y, n_splits=5):
    svr = make_pipeline(StandardScaler(), SVR(C=10.0, epsilon=0.1, kernel="rbf"))
    svr.fit(X[train_idx], y[train_idx])
    pred = svr.predict(X[test_idx])
    svr_rmse.append(np.sqrt(mean_squared_error(y[test_idx], pred)))
    svr_mae.append(mean_absolute_error(y[test_idx], pred))
    svr_r2.append(r2_score(y[test_idx], pred))
metrics["SVR_CV"] = {
    "RMSE": np.mean(svr_rmse),
    "MAE": np.mean(svr_mae),
    "R2": np.mean(svr_r2)
}

# --- XGBoost CV ---
xgb_rmse, xgb_mae, xgb_r2 = [], [], []
for train_idx, test_idx in scaffold_kfold(df, X, y, n_splits=5):
    model = XGBRegressor(**best_params)
    model.fit(X[train_idx], y[train_idx])
    pred = model.predict(X[test_idx])
    xgb_rmse.append(np.sqrt(mean_squared_error(y[test_idx], pred)))
    xgb_mae.append(mean_absolute_error(y[test_idx], pred))
    xgb_r2.append(r2_score(y[test_idx], pred))
metrics["XGBoost_CV"] = {
    "RMSE": np.mean(xgb_rmse),
    "MAE": np.mean(xgb_mae),
    "R2": np.mean(xgb_r2)
}


In [ ]:
# --- Classification labels ---
# Define active if IC50 <= 1 µM (pIC50 >= 6.0)
df["active"] = (df["pIC50"] >= 6.0).astype(int)
y_cls = df["active"].values

# --- RandomForest Classifier CV ---
rf_auc, rf_acc, rf_f1 = [], [], []
for train_idx, test_idx in scaffold_kfold(df, X, y_cls, n_splits=5):
    clf = RandomForestClassifier(n_estimators=500, random_state=42, n_jobs=-1)
    clf.fit(X[train_idx], y_cls[train_idx])
    probas = clf.predict_proba(X[test_idx])[:,1]
    preds = clf.predict(X[test_idx])
    rf_auc.append(roc_auc_score(y_cls[test_idx], probas))
    rf_acc.append(accuracy_score(y_cls[test_idx], preds))
    rf_f1.append(f1_score(y_cls[test_idx], preds))
metrics["RandomForest_Cls"] = {
    "ROC-AUC": np.mean(rf_auc),
    "Accuracy": np.mean(rf_acc),
    "F1": np.mean(rf_f1)
}

# --- SVC CV ---
svr_auc, svr_acc, svr_f1 = [], [], []
for train_idx, test_idx in scaffold_kfold(df, X, y_cls, n_splits=5):
    clf = make_pipeline(StandardScaler(), SVC(C=10.0, kernel="rbf", probability=True))
    clf.fit(X[train_idx], y_cls[train_idx])
    probas = clf.predict_proba(X[test_idx])[:,1]
    preds = clf.predict(X[test_idx])
    svr_auc.append(roc_auc_score(y_cls[test_idx], probas))
    svr_acc.append(accuracy_score(y_cls[test_idx], preds))
    svr_f1.append(f1_score(y_cls[test_idx], preds))
metrics["SVC_Cls"] = {
    "ROC-AUC": np.mean(svr_auc),
    "Accuracy": np.mean(svr_acc),
    "F1": np.mean(svr_f1)
}

# --- XGB Classifier CV ---
xgb_auc, xgb_acc, xgb_f1 = [], [], []
for train_idx, test_idx in scaffold_kfold(df, X, y_cls, n_splits=5):
    clf = XGBClassifier(
        n_estimators=500, learning_rate=0.05, max_depth=6,
        subsample=0.8, colsample_bytree=0.8, random_state=42,
        tree_method="hist", use_label_encoder=False, eval_metric="logloss"
    )
    clf.fit(X[train_idx], y_cls[train_idx])
    probas = clf.predict_proba(X[test_idx])[:,1]
    preds = clf.predict(X[test_idx])
    xgb_auc.append(roc_auc_score(y_cls[test_idx], probas))
    xgb_acc.append(accuracy_score(y_cls[test_idx], preds))
    xgb_f1.append(f1_score(y_cls[test_idx], preds))
metrics["XGBoost_Cls"] = {
    "ROC-AUC": np.mean(xgb_auc),
    "Accuracy": np.mean(xgb_acc),
    "F1": np.mean(xgb_f1)
}

In [ ]:
#GNN Model
print("🧠 Training GNN...")

def mol_to_graph(mol, y_val):
    if mol is None:
        return None
    
    # Node features (atoms)
    x = []
    for a in mol.GetAtoms():
        x.append([
            a.GetAtomicNum(),
            a.GetDegree(),
            a.GetTotalNumHs(),
            a.GetImplicitValence(),
            int(a.GetIsAromatic()),
            a.GetFormalCharge(),
            a.GetHybridization()
        ])
    x = torch.tensor(x, dtype=torch.float)

    # Edge features (bonds)
    ei = [[], []]
    eb = []
    for b in mol.GetBonds():
        i, j = b.GetBeginAtomIdx(), b.GetEndAtomIdx()
        bond_type = b.GetBondTypeAsDouble()
        ei[0] += [i, j]
        ei[1] += [j, i]
        eb.append([bond_type])
        eb.append([bond_type])
    
    edge_index = torch.tensor(ei, dtype=torch.long)
    edge_attr = torch.tensor(eb, dtype=torch.float)

    y_t = torch.tensor([y_val], dtype=torch.float)
    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y_t)

graphs = [mol_to_graph(m,yv) for m,yv in zip(mols,y)]
graphs = [g for g in graphs if g]

# --- GNN uses a single scaffold split ---
train_df, test_df = scaffold_split(df, frac_train=0.8)

train_graphs = [graphs[i] for i in train_df.index]
test_graphs  = [graphs[i] for i in test_df.index]

train_loader = DataLoader(train_graphs, batch_size=32, shuffle=True, generator=g)
test_loader  = DataLoader(test_graphs, batch_size=64, generator=g)

from torch_geometric.nn import GCNConv, global_mean_pool, BatchNorm

class GNNRegressor(nn.Module):
    def __init__(self, in_feats=7, hidden=128, dropout=0.2):
        super().__init__()
        self.conv1 = GCNConv(in_feats, hidden)
        self.bn1 = BatchNorm(hidden)
        self.conv2 = GCNConv(hidden, hidden)
        self.bn2 = BatchNorm(hidden)
        self.lin1 = nn.Linear(hidden, hidden//2)
        self.lin2 = nn.Linear(hidden//2, 1)
        self.dropout = dropout

    def forward(self, x, edge_index, batch, edge_attr=None):
        x = F.relu(self.bn1(self.conv1(x, edge_index)))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = F.relu(self.bn2(self.conv2(x, edge_index)))
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=self.dropout, training=self.training)
        return self.lin2(x).squeeze(-1)

device=torch.device("mps" if torch.backends.mps.is_available() else "cpu")
gnn=GNNRegressor().to(device)
opt=torch.optim.Adam(gnn.parameters(),lr=1e-3)
loss_fn=nn.MSELoss()

EPOCHS = 200
patience = 20
best_loss = float("inf")
patience_counter = 0
#Define the list 
train_losses, val_losses = [], []

for ep in range(EPOCHS):
    # ----- Training -----
    gnn.train(); total=0
    for batch in tqdm(train_loader, desc=f"Epoch {ep+1}/{EPOCHS}"):
        batch = batch.to(device)
        opt.zero_grad()
        pred = gnn(batch.x, batch.edge_index, batch.batch, getattr(batch, "edge_attr", None))
        loss = loss_fn(pred, batch.y)
        loss.backward()
        opt.step()
        total += loss.item()
    avg_train_loss = total / len(train_loader)

    # ----- Validation -----
    gnn.eval(); val_loss=0
    with torch.no_grad():
        for batch in test_loader:
            batch = batch.to(device)
            pred = gnn(batch.x, batch.edge_index, batch.batch, getattr(batch, "edge_attr", None))
            val_loss += loss_fn(pred, batch.y).item()
    avg_val_loss = val_loss / len(test_loader)

    # 🔹 Save for learning curve
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)

    print(f"Epoch {ep+1} | train {avg_train_loss:.4f} | val {avg_val_loss:.4f}")

    # ----- Early Stopping -----
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        patience_counter = 0
        torch.save(gnn.state_dict(), os.path.join(outdir, "best_gnn.pt"))
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("⏹️ Early stopping triggered")
            break

# Load best model
gnn.load_state_dict(torch.load(os.path.join(outdir, "best_gnn.pt")))

gnn.eval(); y_true=[]; y_pred=[]
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating GNN"):
        batch=batch.to(device)
        pred=gnn(batch.x,batch.edge_index,batch.batch)
        y_true+=batch.y.cpu().numpy().tolist()
        y_pred+=pred.cpu().numpy().tolist()
metrics["GNN"]={"RMSE":float(np.sqrt(mean_squared_error(y_true,y_pred))),
                "MAE": float(mean_absolute_error(y_true,y_pred)),
                "R2":  float(r2_score(y_true,y_pred))}

In [ ]:
 #--- GNN Classification ---
print("🧠 Training GNN Classifier...")

# Define binary labels (active if pIC50 >= 6.0)
df["active"] = (df["pIC50"] >= 6.0).astype(int)
y_cls = df["active"].values

# Build graphs for classification
graphs_cls = [mol_to_graph(m, yv) for m, yv in zip(mols, y_cls)]
graphs_cls = [g for g in graphs_cls if g]

# Scaffold split
train_df_cls, test_df_cls = scaffold_split(df, frac_train=0.8)
train_graphs_cls = [graphs_cls[i] for i in train_df_cls.index]
test_graphs_cls  = [graphs_cls[i] for i in test_df_cls.index]

train_loader_cls = DataLoader(train_graphs_cls, batch_size=32, shuffle=True, generator=g)
test_loader_cls  = DataLoader(test_graphs_cls, batch_size=64, generator=g)

# --- GNN Classifier Model ---
class GNNClassifier(nn.Module):
    def __init__(self, in_feats=7, hidden=128, dropout=0.2):
        super().__init__()
        self.conv1 = GCNConv(in_feats, hidden)
        self.bn1 = BatchNorm(hidden)
        self.conv2 = GCNConv(hidden, hidden)
        self.bn2 = BatchNorm(hidden)
        self.lin1 = nn.Linear(hidden, hidden//2)
        self.lin2 = nn.Linear(hidden//2, 1)
        self.dropout = dropout

    def forward(self, x, edge_index, batch, edge_attr=None):
        x = F.relu(self.bn1(self.conv1(x, edge_index)))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = F.relu(self.bn2(self.conv2(x, edge_index)))
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=self.dropout, training=self.training)
        return torch.sigmoid(self.lin2(x)).squeeze(-1)

# Device + optimizer
gnn_cls = GNNClassifier().to(device)
opt_cls = torch.optim.Adam(gnn_cls.parameters(), lr=1e-3)
loss_fn_cls = nn.BCELoss()

# Training loop
EPOCHS = 200
patience = 15
best_loss = float("inf")
patience_counter = 0
train_losses_cls, val_losses_cls = [], []

for ep in range(EPOCHS):
    # Training
    gnn_cls.train(); total=0
    for batch in tqdm(train_loader_cls, desc=f"Epoch {ep+1}/{EPOCHS} (Cls)"):
        batch = batch.to(device)
        opt_cls.zero_grad()
        pred = gnn_cls(batch.x, batch.edge_index, batch.batch, getattr(batch, "edge_attr", None))
        loss = loss_fn_cls(pred, batch.y.float())
        loss.backward()
        opt_cls.step()
        total += loss.item()
    avg_train_loss = total / len(train_loader_cls)

    # Validation
    gnn_cls.eval(); val_loss=0
    with torch.no_grad():
        for batch in test_loader_cls:
            batch = batch.to(device)
            prob = gnn_cls(batch.x, batch.edge_index, batch.batch, getattr(batch, "edge_attr", None))
            val_loss += loss_fn_cls(prob, batch.y.float()).item()
    avg_val_loss = val_loss / len(test_loader_cls)

    # Track
    train_losses_cls.append(avg_train_loss)
    val_losses_cls.append(avg_val_loss)
    print(f"Epoch {ep+1} | train {avg_train_loss:.4f} | val {avg_val_loss:.4f}")

    # Early stopping
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        patience_counter = 0
        torch.save(gnn_cls.state_dict(), os.path.join(outdir, "best_gnn_cls.pt"))
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("⏹️ Early stopping triggered (Cls)")
            break

# Load best model
gnn_cls.load_state_dict(torch.load(os.path.join(outdir, "best_gnn_cls.pt")))


In [ ]:
# Save both regression + classification metrics
with open(os.path.join(outdir, "metrics.json"), "w") as f:
    json.dump(metrics, f, indent=2)

print("✅ Metrics saved:")

# Print regression and classification separately
for model, vals in metrics.items():
    if isinstance(vals, dict):  # check if it's a dictionary
        if "RMSE" in vals and "MAE" in vals:  # regression models
            print(f"{model} → RMSE: {vals['RMSE']:.4f}, MAE: {vals['MAE']:.4f}, R²: {vals['R2']:.4f}")
        elif "ROC-AUC" in vals:  # classification models
            print(f"{model} → ROC-AUC: {vals['ROC-AUC']:.4f}, Accuracy: {vals['Accuracy']:.4f}, F1: {vals['F1']:.4f}")
        else:
            print(f"{model}: {vals}")
    else:
        # vals is just a single number (like mean CV R²)
        print(f"{model}: {vals:.4f}")

df_metrics = pd.DataFrame(metrics).T.reset_index().rename(columns={"index": "Model"})

# --- Evaluation for GNN Classification ---
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, f1_score

gnn_cls.eval(); y_true, y_pred, y_prob = [], [], []
with torch.no_grad():
    for batch in tqdm(test_loader_cls, desc="Evaluating GNN Classifier"):
        batch = batch.to(device)
        prob = gnn_cls(batch.x, batch.edge_index, batch.batch)
        pred = (prob > 0.5).int()
        y_true += batch.y.cpu().numpy().tolist()
        y_pred += pred.cpu().numpy().tolist()
        y_prob += prob.cpu().numpy().tolist()

metrics["GNN_Cls"] = {
    "ROC-AUC": float(roc_auc_score(y_true, y_prob)),
    "PR-AUC": float(average_precision_score(y_true, y_prob)),
    "Accuracy": float(accuracy_score(y_true, y_pred)),
    "F1": float(f1_score(y_true, y_pred))
}
print("✅ GNN Classification Metrics:", metrics["GNN_Cls"])

# --- Plot GNN Classification Metrics ---
plt.figure(figsize=(6,4))
sns.barplot(x=list(metrics["GNN_Cls"].keys()), y=list(metrics["GNN_Cls"].values()))
plt.ylim(0,1)
plt.title("GNN Classification Performance")
plt.tight_layout()
plt.savefig(os.path.join(outdir, "gnn_classification_metrics.png"), dpi=150)
plt.show()

plt.figure(figsize=(7,5))
plt.plot(train_losses_cls, label="Train Loss")
plt.plot(val_losses_cls, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("BCE Loss")
plt.title("GNN Classification Learning Curve")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(outdir, "gnn_cls_learning_curve.png"), dpi=150)
plt.show()

# --- Regression plots (only for models with RMSE/MAE/R2) ---
reg_df = df_metrics.dropna(subset=["R2"], how="any")

if not reg_df.empty:
    plt.figure(figsize=(8,5))
    melt = reg_df.melt(id_vars="Model", value_vars=["RMSE","MAE"], 
                       var_name="Metric", value_name="Value")
    sns.barplot(data=melt, x="Model", y="Value", hue="Metric")
    plt.title("Regression Error Comparison (RMSE vs MAE)")
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, "errors_bar.png"), dpi=150)
    plt.show()

    plt.figure(figsize=(6,4))
    sns.barplot(data=reg_df, x="Model", y="R2")
    plt.title("Regression R² Comparison")
    plt.ylim(0,1)
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, "r2_bar.png"), dpi=150)
    plt.show()

# --- Classification plots (only for models with ROC-AUC/Accuracy/F1) ---
cls_df = df_metrics.dropna(subset=["ROC-AUC"], how="any")

if not cls_df.empty:
    plt.figure(figsize=(8,5))
    melt = cls_df.melt(id_vars="Model", value_vars=["Accuracy","F1"], 
                       var_name="Metric", value_name="Value")
    sns.barplot(data=melt, x="Model", y="Value", hue="Metric")
    plt.title("Classification Accuracy / F1 Comparison")
    plt.ylim(0,1)
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, "cls_acc_f1.png"), dpi=150)
    plt.show()

    plt.figure(figsize=(6,4))
    sns.barplot(data=cls_df, x="Model", y="ROC-AUC")
    plt.title("Classification ROC-AUC Comparison")
    plt.ylim(0,1)
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, "cls_auc.png"), dpi=150)
    plt.show()

# 🔹 Plot training vs validation loss curve
plt.figure(figsize=(7,5))
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("GNN Learning Curve")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(outdir,"gnn_learning_curve.png"), dpi=150)
plt.show()
